# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846820221274                   -0.70    4.8         
  2   -7.852314995646       -2.26       -1.53    1.0   24.1ms
  3   -7.852614473017       -3.52       -2.55    1.5   26.5ms
  4   -7.852646019421       -4.50       -2.90    2.8   33.7ms
  5   -7.852646532473       -6.29       -3.23    1.0   24.4ms
  6   -7.852646679003       -6.83       -4.01    1.0   24.3ms
  7   -7.852646686514       -8.12       -5.23    1.8   28.9ms
  8   -7.852646686723       -9.68       -5.40    2.0   30.1ms
  9   -7.852646686728      -11.28       -5.78    1.2   25.5ms
 10   -7.852646686730      -11.85       -6.86    1.0   24.6ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846791197410                   -0.70           4.5         
  2   -7.852525664830       -2.24       -1.63   0.80    2.0    256ms
  3   -7.852636693175       -3.95       -2.72   0.80    1.0   22.2ms
  4   -7.852646456156       -5.01       -3.31   0.80    2.2   29.5ms
  5   -7.852646682603       -6.65       -4.16   0.80    1.8   26.4ms
  6   -7.852646686398       -8.42       -4.85   0.80    1.2   25.0ms
  7   -7.852646686722       -9.49       -5.82   0.80    1.8   26.9ms
  8   -7.852646686730      -11.13       -6.74   0.80    2.5   30.5ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.415700e+01     3.702282e+00
 * time: 0.438647985458374
     1     1.008384e+00     1.890780e+00
 * time: 0.7487719058990479
     2    -1.589538e+00     2.251019e+00
 * time: 0.771824836730957
     3    -3.868164e+00     1.965930e+00
 * time: 0.8046817779541016
     4    -5.036559e+00     1.691528e+00
 * time: 0.8375508785247803
     5    -6.868612e+00     1.013069e+00
 * time: 0.8704788684844971
     6    -7.505920e+00     4.842897e-01
 * time: 0.9037399291992188
     7    -7.712201e+00     1.629385e-01
 * time: 0.9267477989196777
     8    -7.788790e+00     1.893895e-01
 * time: 0.9499619007110596
     9    -7.825465e+00     6.750482e-02
 * time: 0.9731488227844238
    10    -7.838379e+00     5.507646e-02
 * time: 0.9964158535003662
    11    -7.843108e+00     5.387135e-02
 * time: 1.0196897983551025
    12    -7.844606e+00     5.398411e-02
 * time: 1.0429978370666504
    13    -7.846563e+00     3.791053e-02
 * time: 1.066438913345

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846939022233                   -0.70    4.5         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645941520                   -1.64         
  2   -7.852646686730       -6.13       -3.71    2.30s
  3   -7.852646686730      -13.36       -7.27    127ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 1.2546271372765062e-7
|ρ_newton - ρ_scfv| = 1.1307107299869511e-7
|ρ_newton - ρ_dm|   = 1.3063831290243122e-9
